# Identify H3N2 epitope sites and average escape scores from antigenic escape scores per serum, site, and amino acid

Identifies contemporary epitope sites and average escape scores from experimental measurements. Antigenic escape scores were originally calculated in Welsh et al. 2023 per serum a range of age groups. In this notebook, we identify putative antigenic sites or "epitope sites" where mutations should allow recent H3N2 strains to escape existing immunity. These sites augment the historical epitope sites from Wolf et al. 2006 that Nextstrain seasonal influenza analyses use to calculate "epitope mutations" in each HA tree.

In addition to finding epitope sites from experimental data, we also calculate the average non-negative antigenic escape score per HA1 site/position and amino acid mutation from the wildtype. We save these average scores as [an Augur "distance map"](https://docs.nextstrain.org/projects/augur/en/stable/usage/cli/distance.html) which allows us to calculate a cumulative escape score for all HA1 amino acid mutations per strain in a given HA tree. We use non-negative escape scores to reflect the assumption that mutations with negative escape scores are less likely to occur in nature.

## Import

In [ ]:
import json
import pandas as pd

In [ ]:
ha1_amino_acids = 329

Define the number of standard deviations from the mean to use for the threshold of putative epitope sites.

In [ ]:
n_std_dev = 4

## Load data

Load data from the merged and filtered escape scores in [the Welsh et al. GitHub repository](https://github.com/dms-vep/flu_h3_hk19_dms.git).

In [ ]:
escape_scores = pd.read_csv("https://github.com/dms-vep/flu_h3_hk19_dms/raw/main/results/full_hk19_escape_scores.csv")

In [ ]:
escape_scores.shape

In [ ]:
escape_scores.head()

In [ ]:
escape_scores["serum"].drop_duplicates().shape

In [ ]:
set(escape_scores["serum"].drop_duplicates().values)

In [ ]:
nonnegative_ha1_escape_scores = escape_scores.query(f"(escape_mean >= 0) & (site > 0) & (site <= {ha1_amino_acids})").copy()

In [ ]:
nonnegative_ha1_escape_scores.shape

In [ ]:
nonnegative_ha1_escape_scores.head()

In [ ]:
nonnegative_ha1_escape_scores.tail()

Sum non-negative escape scores per serum and site, calculate the mean and std dev, and identify putative epitope sites as those where the total escape score is greater than the mean plus 4 standard deviations.

In [ ]:
ha1_effects_per_serum_and_site = nonnegative_ha1_escape_scores.groupby([
    "serum",
    "cohort",
    "site",
])["escape_mean"].sum().reset_index().rename(columns={"escape_mean": "escape_total"})

In [ ]:
ha1_effects_per_serum_and_site.shape

In [ ]:
ha1_effects_per_serum_and_site.head()

In [ ]:
ha1_effects_per_serum = ha1_effects_per_serum_and_site.groupby(
    "serum"
).agg({
    "escape_total": ["mean", "std"]
}).reset_index().set_axis(["serum", "serum_mean_escape_total", "serum_std_escape_total"], axis=1)

In [ ]:
ha1_effects_per_serum.shape

In [ ]:
ha1_effects_per_serum.head()

In [ ]:
ha1_effects_per_serum["serum_escape_threshold"] = (
    ha1_effects_per_serum["serum_mean_escape_total"] + (n_std_dev * ha1_effects_per_serum["serum_std_escape_total"])
)

In [ ]:
ha1_effects_per_serum.head()

In [ ]:
ha1_effects_per_serum_and_site_with_threshold = ha1_effects_per_serum_and_site.merge(ha1_effects_per_serum, on="serum")

In [ ]:
ha1_effects_per_serum_and_site_with_threshold.head()

In [ ]:
epitope_sites_by_serum = ha1_effects_per_serum_and_site_with_threshold.query("escape_total >= serum_escape_threshold").copy()

In [ ]:
epitope_sites_by_serum.head()

In [ ]:
epitope_site_counts = epitope_sites_by_serum.groupby("site")["serum"].count()

In [ ]:
epitope_site_counts

In [ ]:
epitope_sites = epitope_site_counts.index.values

In [ ]:
epitope_sites

In [ ]:
len(epitope_sites)

Export a simple distance map for epitope sites where each site identified above has a weight of 1. When used with augur distance, this map calculates the Hamming distance between each sample and the MRCA of the tree at only these positions. All other mutations get ignored in that distance calculation.

In [ ]:
epitope_site_distance_map = {
    "name": "Welsh et al. epitope sites",
    "default": 0,
    "map": {
        "HA1": {}
    }
}

In [ ]:
for site in epitope_sites:
    epitope_site_distance_map["map"]["HA1"][str(site)] = 1

In [ ]:
epitope_site_distance_map

In [ ]:
with open("welsh_epitope_sites.json", "w") as oh:
    json.dump(
        epitope_site_distance_map,
        oh,
        indent=2,
    )

## Calculate average nonnegative scores per site and amino acid across all samples

In [ ]:
total_score_per_site_and_amino_acid = {}

In [ ]:
count_score_per_site_and_amino_acid = {}

In [ ]:
nonnegative_ha1_escape_scores.head()

In [ ]:
mean_escape_score_by_site_and_amino_acid = nonnegative_ha1_escape_scores.groupby([
    "site",
    "wildtype",
    "mutant",
])["escape_mean"].mean().to_dict()

Export a per-site-and-amino-acid distance map for the average scores calculated above. When used with augur distance, this map will calculate a weighted Hamming distance between each sample and the MRCA of the tree at any site with a mutation specifically from the wild type allele to the experimentally measured allele. The weights of the Hamming distance are the average escape score values for each site and amino acid mutation.

In [ ]:
distance_map = {
    "name": "Welsh et al. escape scores per site and amino acid",
    "default": 0,
    "map": {
        "HA1": {}
    }
}

In [ ]:
for (site, wildtype, mutant), escape_score in mean_escape_score_by_site_and_amino_acid.items():
    if str(site) not in distance_map["map"]["HA1"]:
        distance_map["map"]["HA1"][str(site)] = []

    distance_map["map"]["HA1"][str(site)].append({
        "from": wildtype,
        "to": mutant,
        "weight": round(escape_score, 6),
    })

In [ ]:
distance_map["map"]["HA1"]["160"]

In [ ]:
distance_map["map"]["HA1"]["223"]

In [ ]:
distance_map["map"]["HA1"]["140"]

In [ ]:
with open("welsh_escape_by_site_and_amino_acid.json", "w") as oh:
    json.dump(
        distance_map,
        oh,
        indent=2,
    )